In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

plt.rcdefaults()  # reset to default
plt.style.use('phys-plots.mplstyle')

In [26]:
x, y, z, w0, k = sp.symbols('x,y,z,w0,k', real=True)
mm = sp.symbols('mm')

alpha_e, alpha_m, alpha_Qe, alpha_Qm = sp.symbols('alpha_e, alpha_m, alpha_Qe, alpha_Qm', real=True)

# A = 0
alpha_e = 0
alpha_m = 0

# C = 0
# alpha_Qm = -2*alpha_Qe

# B = 0
alpha_Qe = -2*alpha_Qm


IMalpha_e, IMalpha_m, IMalpha_Qe, IMalpha_Qm = sp.symbols('IMalpha_e, IMalpha_m, IMalpha_Qe, IMalpha_Qm', real=True)

epsilon, mu, A = sp.symbols('epsilon, mu, A', positive=True)

E0 = A/sp.sqrt(epsilon)
H0 = A/sp.sqrt(mu)

# alpha_e = sp.I * IMalpha_e
# alpha_m = sp.I * IMalpha_m
# alpha_Qe = sp.I * IMalpha_Qe
# alpha_Qm = sp.I * IMalpha_Qm
#alpha_Qm

coords = [x, y, z]

i, j, p = sp.symbols('i, j, p', integer=True)

#w0 = 1
mm = 0

z0 = k * w0**2 / 2
w = w0 * sp.sqrt(1 + z**2 / z0**2)
Rinv = z / (z**2 + z0**2)

eta = sp.atan2(z, z0)

#gauss_parax = w0/w * sp.exp( -(x**2 + y**2)/w**2 ) * sp.exp(1j * (k * z - eta + k*(x**2 + y**2) / 2 * Rinv)) 
gauss_parax = sp.exp( -(x**2 + y**2)/w0**2 )

ux = 1 / sp.sqrt(1 + sp.Abs(mm)**2)
uy = mm / sp.sqrt(1 + sp.Abs(mm)**2)


E = [E0 * ux * gauss_parax, E0 * uy * gauss_parax, 0]

umagpol = [-uy, ux, 0]
H = [H0 * umagpol[0] * gauss_parax, H0 * umagpol[1] * gauss_parax, 0]


def eps(i, j, k):
    return sp.LeviCivita(i, j, k)


e = [epsilon*alpha_e * E[0], epsilon*alpha_e * E[1], epsilon*alpha_e * E[2]]
m = [mu*alpha_m * H[0], mu*alpha_m * H[1], mu*alpha_m * H[2]]

Qe = [
    [epsilon*alpha_Qe * 1/2 * (sp.diff(E[i], coords[j]) + sp.diff(E[j], coords[i]))
     for i in range(3)]
     for j in range(3)
]

Qm = [
    [mu*alpha_Qm * 1/2 * (sp.diff(H[i], coords[j]) + sp.diff(H[j], coords[i]))
     for i in range(3)]
     for j in range(3)
]


# ∇_i E_j
grad_E = [[sp.diff(E[j], coords[i]) 
      for j in range(3)]
      for i in range(3)]

# ∇_i H_j
grad_H = [[sp.diff(H[j], coords[i]) 
      for j in range(3)]
      for i in range(3)]


# ∇_i ∇_j E_p
grad_grad_E = [[[sp.diff(E[p], coords[i], coords[j]) for p in range(3)]
      for j in range(3)]
      for i in range(3)]

# ∇_i ∇_j H_p
grad_grad_H = [[[sp.diff(H[p], coords[i], coords[j]) for p in range(3)]
      for j in range(3)]
      for i in range(3)]



# Electric dipole force
# Fe_i = Re[e_j^* ∇_i E_j]
Fe = [sp.re(sum(sp.conjugate(e[j]) * grad_E[i][j] for j in range(3))) for i in range(3)]

# Magnetic dipole force
# Fm_i = Re[m_j^* ∇_i H_j]
Fm = [sp.re(sum(sp.conjugate(m[j]) * grad_H[i][j] for j in range(3))) for i in range(3)]

# Electric-magnetic recoil
# Fem_i = -k^4/(12π) Re[ε_ijk e_j^* m_k]
Fem = [-k**4/(12*sp.pi) * sp.re(sum(eps(i, j, k) * sp.conjugate(e[j]) * m[k] for j in range(3) for k in range(3))) for i in range(3)]

# Electric dipole-quadrupole 
# FeQe_i = k^5/(40π) Im[Qe_ij^* e_j]
FeQe = [k**5/(40*sp.pi) * sp.im(sum(sp.conjugate(Qe[i][j]) * e[j] for j in range(3))) for i in range(3)]

# Magentic dipole-quadrupole 
# FmQm_i = k^5/(40π) Im[Qm_ij^* m_j]
FmQm = [k**5/(40*sp.pi) * sp.im(sum(sp.conjugate(Qm[i][j]) * m[j] for j in range(3))) for i in range(3)]


# Quadrupole-quadrupole
# FQeQm_i = -k^6/(250π) Re[ε_ijk Qe_lj^* Qm_lk]
FQeQm = [-k**6/(250*sp.pi) * sp.re(sum(eps(i, j, k) * sp.conjugate(Qe[l][j]) * Qm[l][k] for j in range(3) for k in range(3) for l in range(3))) for i in range(3)]

# Electric quadrupole force
# FQe_i = 1/4 Re[Qe_jk^* ∇_i ∇_k E_j]
FQe = [1/4 * sp.re(sum(sp.conjugate(Qe[j][k]) * grad_grad_E[i][k][j] for j in range(3) for k in range(3))) for i in range(3)]

# Magnetic quadrupole force
# FQm_i = 1/4 Re[Qm_jk^* ∇_i ∇_k H_j]
FQm = [1/4 * sp.re(sum(sp.conjugate(Qm[j][k]) * grad_grad_H[i][k][j] for j in range(3) for k in range(3))) for i in range(3)]

In [28]:
AA = 2/w0**3 * (alpha_e + alpha_m)
BB = (alpha_Qe + 2*alpha_Qm) / w0**5
CC = (2*alpha_Qe + alpha_Qm) / w0**5


sp.simplify((Fe[0] + Fm[0] + FQe[0] + FQm[0])/(x/w0 * U0/w0 * CC * sp.exp( -2*(x**2 + y**2)/w0**2 )))

0.5 - 1.0*x**2/w0**2

In [14]:
U0 = w0**3 * A**2

AA = 2/w0**3 * (alpha_e + alpha_m)
AA, BB, CC = sp.symbols('AA BB CC')

In [15]:
expr = sp.simplify((Fe[0] + Fm[0] + FQe[0] + FQm[0])/(U0 * sp.exp( -2*(x**2 + y**2)/w0**2 )))
expr

1.0*x*(1.0*alpha_Qe*w0**2 - 2.0*alpha_Qe*x**2 - 1.0*alpha_Qe*y**2 + 0.5*alpha_Qm*w0**2 - 1.0*alpha_Qm*x**2 - 2.0*alpha_Qm*y**2 - 2.0*alpha_e*w0**4 - 2.0*alpha_m*w0**4)/w0**9

In [17]:
expr.subs((alpha_e + alpha_m), AA)

1.0*x*(1.0*alpha_Qe*w0**2 - 2.0*alpha_Qe*x**2 - 1.0*alpha_Qe*y**2 + 0.5*alpha_Qm*w0**2 - 1.0*alpha_Qm*x**2 - 2.0*alpha_Qm*y**2 - 2.0*alpha_e*w0**4 - 2.0*alpha_m*w0**4)/w0**9